# MCI customer churn prediction


In [1]:
import pandas as pd
import numpy as np

# import matplotlib.pyplot as plt

## Data prepration

In [2]:
ls

Customer_Churn.csv*  mci_churn.ipynb*


In [3]:
data = 'Customer_Churn.csv'

In [31]:
df = pd.read_csv(data)

## EDA & Setting up validation framework

### data exploration

In [5]:
df.head().T

,0,1,2,3,4
Call Failure,8.00,0.000,10.00,10.00,3.000
Complains,0.00,0.000,0.00,0.00,0.000
Subscription Length,38.00,39.000,37.00,38.00,38.000
Charge Amount,0.00,0.000,0.00,0.00,0.000
Seconds of Use,4370.00,318.000,2453.00,4198.00,2393.000
Frequency of use,71.00,5.000,60.00,66.00,58.000
Frequency of SMS,5.00,7.000,359.00,1.00,2.000
Distinct Called Numbers,17.00,4.000,24.00,35.00,33.000
Age Group,3.00,2.000,3.00,1.00,1.000
Tariff Plan,1.00,1.000,1.00,1.00,1.000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3150 entries, 0 to 3149
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Call  Failure            3150 non-null   int64  
 1   Complains                3150 non-null   int64  
 2   Subscription  Length     3150 non-null   int64  
 3   Charge  Amount           3150 non-null   int64  
 4   Seconds of Use           3150 non-null   int64  
 5   Frequency of use         3150 non-null   int64  
 6   Frequency of SMS         3150 non-null   int64  
 7   Distinct Called Numbers  3150 non-null   int64  
 8   Age Group                3150 non-null   int64  
 9   Tariff Plan              3150 non-null   int64  
 10  Status                   3150 non-null   int64  
 11  Age                      3150 non-null   int64  
 12  Customer Value           3150 non-null   float64
 13  Churn                    3150 non-null   int64  
dtypes: float64(1), int64(13)

In [32]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [33]:
df.churn.unique()

array([0, 1])

In [9]:
df.isnull().sum()

call__failure              0
complains                  0
subscription__length       0
charge__amount             0
seconds_of_use             0
frequency_of_use           0
frequency_of_sms           0
distinct_called_numbers    0
age_group                  0
tariff_plan                0
status                     0
age                        0
customer_value             0
churn                      0
dtype: int64

In [10]:
any(df.dtypes == 'object')

False

### Making Train Test Validation sets

In [12]:
from sklearn.model_selection import train_test_split

In [35]:
df_full_train , df_test = train_test_split(df , test_size=.2 , random_state=42)

In [14]:
df_train , df_val = train_test_split(df_full_train , test_size=.25 , random_state=42)

In [15]:
len(df_train), len(df_test) , len(df_val)

(1890, 630, 630)

In [18]:
y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

In [30]:
del df_train['churn']
del df_val['churn']
del df_test['churn']

### feature importance 

#### global churn balance 

In [73]:
# we can use mean() to calculate rate because we just have a 'uniform distribution'.

global_churn_rate = df_full_train.churn.mean()
round(float(global_churn_rate), 2)

0.15

In [41]:
df_full_train.churn.value_counts(), df_full_train.churn.value_counts(normalize=True)

(churn
 0    2135
 1     385
 Name: count, dtype: int64,
 churn
 0    0.847222
 1    0.152778
 Name: proportion, dtype: float64)

In [68]:
# we see that features age and age_group are the same 
df.age_group.unique() , df.age.unique(), df[df.age_group == 1].age.unique()

(array([3, 2, 1, 4, 5]), array([30, 25, 15, 45, 55]), array([15]))

#### churn rate in age groups

In [100]:
for i in sorted(df.age_group.unique()) :
    print( round((df.age_group == i ).mean(), 2), '   ', end='' )

0.04    0.33    0.45    0.13    0.05    

In [119]:
for col in df.columns:
    print('{}: {}  {}'.format(col , df[col].nunique(), '   '), end='')

call__failure: 37     complains: 2     subscription__length: 45     charge__amount: 11     seconds_of_use: 1756     frequency_of_use: 242     frequency_of_sms: 405     distinct_called_numbers: 92     age_group: 5     tariff_plan: 2     status: 2     age: 5     customer_value: 2654     churn: 2     

In [128]:
df[df.tariff_plan == 1].churn.value_counts(normalize=True)

churn
0    0.83167
1    0.16833
Name: proportion, dtype: float64

In [130]:
df.columns

Index(['call__failure', 'complains', 'subscription__length', 'charge__amount',
       'seconds_of_use', 'frequency_of_use', 'frequency_of_sms',
       'distinct_called_numbers', 'age_group', 'tariff_plan', 'status', 'age',
       'customer_value', 'churn'],
      dtype='object')